In [1]:
%load_ext autoreload
%autoreload 2


import time
import joblib
import random
import clipboard
import numpy as np
from itertools import repeat
from operator import attrgetter
from deap import base, tools, creator, algorithms
try:
    from collections.abc import Sequence
except ImportError:
    from collections import Sequence

# GA

In [2]:
def get_boiler_inflection_points(num_radiators, max_energy_consumption_coefficient):
  boiler_inflection_point_list = [
    float(num_radiators * max_energy_consumption_coefficient * (1 / 6)),
    float(num_radiators * max_energy_consumption_coefficient * (1 / 2))
  ]
  return np.array(boiler_inflection_point_list)


def get_temperature_prediction_coefficients(num_radiators):
  temp_pred_coef_list = [[-.175469, 0, 15.095, 15.49728, 15.89319]] * num_radiators # 12.01728
  return np.array(temp_pred_coef_list).T


def calculate_delta_temp_list(
  curr_temp_list, 
  target_temp_list
):
  return target_temp_list - curr_temp_list


def set_modulation_list(
  curr_temp_list, 
  target_temp_list, 
  valve_modulation_limit_list
):
  delta_temp_list = calculate_delta_temp_list(curr_temp_list, target_temp_list)
  modulations = []
  for i in range(delta_temp_list.shape[0]):
    delta_temp = delta_temp_list[i]

    if delta_temp <= valve_modulation_limit_list[0]:
      modulations.append(0)
    elif valve_modulation_limit_list[0] <= delta_temp <= valve_modulation_limit_list[1]:
      modulations.append(1)
    elif valve_modulation_limit_list[1] <= delta_temp <= valve_modulation_limit_list[2]:
      modulations.append(2)
    elif valve_modulation_limit_list[2] <= delta_temp <= 10:
      modulations.append(3)
  return np.array(modulations)


def calculate_energy_consumption_list(
  valve_modulation_energy_consumption_list, 
  valve_modulation_list
):
  return valve_modulation_energy_consumption_list[valve_modulation_list]


def predict_temperature_with_regression(
  target_temp_list, 
  curr_temp_list, 
  demand_total_time, 
  supply_granularity, 
  valve_modulation_limit_list, 
  temperature_prediction_coef_list, 
  num_radiators, 
  valve_modulation_energy_consumption_list
):
  hum_pred = 44
  iter_curr_temp_list = curr_temp_list.copy()
  temperature_prediction_list = list()
  energy_consumption_list = list()
  for t in range(demand_total_time):
    target_temp_list_at_t = target_temp_list[:, int(t / supply_granularity)]
    
    if any([i not in [0, 1, 2, 3] for i in target_temp_list_at_t]):
      valve_modulation_list = set_modulation_list(
        iter_curr_temp_list, 
        target_temp_list_at_t, 
        valve_modulation_limit_list
      )
    else:
      valve_modulation_list = target_temp_list_at_t
    
    temp_prediction_coef_list_over_hum_pred = temperature_prediction_coef_list[1:5] / hum_pred
    temp_diff_pred = np.array(
      [
        temp_prediction_coef_list_over_hum_pred[valve_modulation_list[f], f] 
        for f in range(num_radiators)
      ]
    ) + temperature_prediction_coef_list[0] / hum_pred
    iter_curr_temp_list += temp_diff_pred
    temperature_prediction_list.append(iter_curr_temp_list.copy())
    valve_energy_consumption_list = calculate_energy_consumption_list(
      valve_modulation_energy_consumption_list, 
      valve_modulation_list
    )
    energy_consumption_list.append(valve_energy_consumption_list.copy())
  return np.array(temperature_prediction_list).T, np.array(energy_consumption_list).T


def generate_individual(
  supply_total_time
):
  ind = np.random.choice(
    [0, 1, 2, 3], 
    p = [0.4, 0.3, 0.2, 0.1],
    size=(supply_total_time, )
  )
  return ind


def crossover(
  ind1,
  ind2,
  cx_apartment_proba,
  cx_time_point_proba
):
  apartment_length, time_length = ind1.shape
  
  for i in range(apartment_length):
    if random.random() < cx_apartment_proba:
      if random.random() < 0.2:
        indices = random.sample(range(time_length), 2)
        t0 = min(indices)
        t1 = max(indices)

        ind1_val = ind1[i][t0 : t1]
        ind2_val = ind2[i][t0 : t1]

        ind1[i][t0 : t1] = ind2_val
        ind2[i][t0 : t1] = ind1_val
      else:
        for j in range(time_length):
          if random.random() < cx_time_point_proba:
            ind1_val = ind1[i][j]
            ind2_val = ind2[i][j]

            ind1[i][j] = ind2_val
            ind2[i][j] = ind1_val

  return ind1, ind2


def mutate(
  individual, 
  lower_limit, 
  upper_limit, 
  mutation_apartment_proba,
  mutation_time_point_proba
):
  apartment_length, time_length = individual.shape
    
  for i in range(apartment_length):
    if random.random() < mutation_apartment_proba:
      for j in range(time_length):
        if random.random() < mutation_time_point_proba:
          val = individual[i][j]
          
          if random.random() < 0.2:
            new_val = np.random.choice(
              [0, 1, 2, 3],
              p = [0.4, 0.3, 0.2, 0.1]
            )
          else:
            if val == lower_limit:
              new_val = lower_limit + 1
            elif val == upper_limit:
              new_val = upper_limit - 1
            else:
              new_val = val + np.random.choice([-1, 1])

          individual[i][j] = new_val
  return individual,


def selection(
  individuals, 
  k, 
  fit_attr="fitness"
):
  best_selection_alpha = 0.8
  sorted_individuals = sorted(individuals, key=attrgetter(fit_attr), reverse=True)

  selected_best = tools.selBest(
    individuals,
    int(k * best_selection_alpha), 
    fit_attr=fit_attr
  )
  selected_random = tools.selRandom(
    sorted_individuals[int(len(individuals) * best_selection_alpha) :], 
    int(k * (1 - best_selection_alpha))
  )
  return selected_best + selected_random


def calculate_unmet_demand(
  temperature_prediction_list, 
  demanded_temperature_list
):
  return temperature_prediction_list - demanded_temperature_list


def calculate_rmse(
  li
):
  return np.sqrt((li ** 2).mean())


def calculate_custom_score(
  li
):
  temp_li = np.abs(li[li < 0])
  return ((10 + 100 * temp_li) ** 2).sum()


def calculate_negative_unmet_demand(
  unmet_demand_list
):
  negative_unmet_demand_list = unmet_demand_list.copy()
  negative_unmet_demand_list[negative_unmet_demand_list > 0] = 0.0
  return negative_unmet_demand_list


def evaluate_unmet_demand_one_way(
  unmet_demand_list
):
  negative_unmet_demand_list = calculate_negative_unmet_demand(unmet_demand_list)
  return calculate_custom_score(negative_unmet_demand_list)


def evaluate_unmet_demand_two_way(
  unmet_demand_list
):
  return calculate_rmse(unmet_demand_list)


def evaluate_energy_consumption(
  energy_consumption_list, 
  boiler_inflection_point_list, 
  boiler_cost_list
):
  total_energy_consumption_list = energy_consumption_list.sum(axis=0)
  boiler_energy_consumption_cost_list = np.where(
    total_energy_consumption_list <= boiler_inflection_point_list[0], 
    boiler_cost_list[0],
    np.where(
      total_energy_consumption_list <= boiler_inflection_point_list[1], 
      boiler_cost_list[1], 
      boiler_cost_list[2]
    )
  )
  
#   boiler_energy_consumption_cost_list **= 10
  return boiler_energy_consumption_cost_list.sum()


def evaluate(
  target_temp_list, 
  curr_temp_list, 
  demand_total_time, 
  supply_granularity, 
  valve_modulation_limit_list, 
  temperature_prediction_coef_list, 
  num_radiators, 
  valve_modulation_energy_consumption_list, 
  demanded_temperature_list,
  boiler_inflection_point_list, 
  boiler_cost_list
):
  temperature_prediction_list, energy_consumption_list = predict_temperature_with_regression(
    target_temp_list, 
    curr_temp_list, 
    demand_total_time, 
    supply_granularity,
    valve_modulation_limit_list, 
    temperature_prediction_coef_list, 
    num_radiators,
    valve_modulation_energy_consumption_list
  )

  unmet_demand_list = calculate_unmet_demand(
    temperature_prediction_list, 
    demanded_temperature_list
  )
  demand_evaluation = evaluate_unmet_demand_one_way(unmet_demand_list)

  energy_consumption_evaluation = evaluate_energy_consumption(
    energy_consumption_list, 
    boiler_inflection_point_list, 
    boiler_cost_list
  )
  
  score = demand_evaluation + energy_consumption_evaluation

  return score,


creator.create("FitnessMin", base.Fitness, weights=(-1.0, ))
creator.create("Individual", np.ndarray, fitness=creator.FitnessMin)


def genetic_solver_run(
  num_radiators, 
  population_size, 
  offspring_size, 
  crossover_proba, 
  cx_apartment_proba,
  cx_time_point_proba,
  mutation_proba, 
  mutation_apartment_proba, 
  mutation_time_point_proba,
  demand_granularity, 
  supply_granularity, 
  application_period, 
  data_unit_time, 
  generation_length, 
  valve_min_modulation,
  valve_max_modulation, 
  current_temperature_list, 
  demanded_temperature_list, 
  valve_modulation_limit_list, 
  valve_modulation_energy_consumption_list, 
  boiler_inflection_point_list, 
  boiler_cost_list,
  temperature_prediction_coef_list
):
  res_dict = dict()
  
  demand_total_time = int(application_period * demand_granularity)
  supply_total_time = int(application_period * demand_granularity / supply_granularity)
  
  demanded_temperature_list = np.repeat(
    demanded_temperature_list, 
    demand_granularity
  ).reshape(num_radiators, -1)
  
  res_dict["demanded_temperature_list"] = demanded_temperature_list
  
  current_practice_schedule_list = demanded_temperature_list

  current_practice_schedule_evaluation = evaluate(
    current_practice_schedule_list, 
    curr_temp_list=current_temperature_list, 
    demand_total_time=demand_total_time,
    supply_granularity=supply_granularity, 
    valve_modulation_limit_list=valve_modulation_limit_list, 
    temperature_prediction_coef_list=temperature_prediction_coef_list, 
    num_radiators=num_radiators,
    valve_modulation_energy_consumption_list=valve_modulation_energy_consumption_list, 
    demanded_temperature_list=demanded_temperature_list,
    boiler_inflection_point_list=boiler_inflection_point_list, 
    boiler_cost_list=boiler_cost_list
  )
  (
    current_practice_temperature_prediction_list, 
    current_practice_energy_consumption_list
  ) = predict_temperature_with_regression(
    current_practice_schedule_list,
    current_temperature_list, 
    demand_total_time, 
    supply_granularity, 
    valve_modulation_limit_list, 
    temperature_prediction_coef_list,
    num_radiators, 
    valve_modulation_energy_consumption_list
  )
  current_practice_unmet_demand_list = calculate_unmet_demand(
    current_practice_temperature_prediction_list, 
    demanded_temperature_list
  )
  current_practice_total_energy_consumption_list = current_practice_energy_consumption_list.sum(axis=0)
  current_practice_boiler_energy_consumption_cost_list = np.where(
    current_practice_total_energy_consumption_list <= boiler_inflection_point_list[0],
    boiler_cost_list[0],
    np.where(
      current_practice_total_energy_consumption_list <= boiler_inflection_point_list[1], 
      boiler_cost_list[1], 
      boiler_cost_list[2]
    )
  )
  
  res_dict["current_practice"] = {
    "evaluation_result": current_practice_schedule_evaluation[0],
    "negative_unmet_demand_mean": calculate_negative_unmet_demand(current_practice_unmet_demand_list).mean(),
    "energy_consumption_sum": current_practice_boiler_energy_consumption_cost_list.sum(),
    "energy_consumption_mean": current_practice_boiler_energy_consumption_cost_list.mean(),
    "list": current_practice_schedule_list,
    "temperature_prediction_list": current_practice_temperature_prediction_list,
    "unmet_demand_list": current_practice_unmet_demand_list,
    "energy_consumption_list": current_practice_boiler_energy_consumption_cost_list
  }
  
  toolbox = base.Toolbox()
  toolbox.register(
    "generate_individual", 
    generate_individual, 
    supply_total_time=supply_total_time
  )
  toolbox.register(
    "mate", 
    crossover,
    cx_apartment_proba = cx_apartment_proba,
    cx_time_point_proba = cx_time_point_proba
  )
  toolbox.register(
    "mutate", 
    mutate, 
    lower_limit = valve_min_modulation, 
    upper_limit = valve_max_modulation,
    mutation_apartment_proba = mutation_apartment_proba,
    mutation_time_point_proba = mutation_time_point_proba
  )
  toolbox.register(
    "select", 
    selection
  )

  stats = tools.Statistics(lambda ind: ind.fitness.values)
  stats.register("avg", np.mean)
  stats.register("std", np.std)
  stats.register("min", np.min)
  stats.register("max", np.max)

  start_time = time.time()
  pop_list = []
  res_dict["internal_logs"] = []
  i = 0
  while i < num_radiators:
    if num_radiators - i > 4 or (num_radiators - i) % 3 == 0:
      num = 3
    else:
      num = 2
      
    print(f"Radiators {', '.join([str(j) for j in range(i + 1, i + num + 1)])}")

    toolbox.register(
      "individual", 
      tools.initRepeat,
      container=creator.Individual, 
      func=toolbox.generate_individual, 
      n=num
    )
    toolbox.register(
      "population", 
      tools.initRepeat, 
      container=list, 
      func=toolbox.individual
    )
    toolbox.register(
      "evaluate", 
      evaluate, 
      curr_temp_list=current_temperature_list[i : i + num], 
      demand_total_time=demand_total_time, 
      supply_granularity=supply_granularity,
      valve_modulation_limit_list=valve_modulation_limit_list, 
      temperature_prediction_coef_list=temperature_prediction_coef_list[:, i : i + num], 
      num_radiators=num,
      valve_modulation_energy_consumption_list=valve_modulation_energy_consumption_list, 
      demanded_temperature_list=demanded_temperature_list[i : i + num, :],
      boiler_inflection_point_list=boiler_inflection_point_list * num / num_radiators, 
      boiler_cost_list=boiler_cost_list
    )

    internal_pop = toolbox.population(n=population_size)
    
    internal_pop, internal_logs = algorithms.eaMuPlusLambda(
      internal_pop, 
      toolbox, 
      mu=population_size, 
      lambda_=offspring_size, 
      cxpb=crossover_proba, 
      mutpb=mutation_proba, 
      ngen=generation_length,
      stats=stats, 
      verbose=False
    )

    pop_list.append(internal_pop)
    res_dict["internal_logs"].append(internal_logs)

    i += num
    
  toolbox.register(
    "individual", 
    tools.initRepeat,
    container=creator.Individual, 
    func=toolbox.generate_individual, 
    n=num_radiators
  )
  toolbox.register(
    "population", 
    tools.initRepeat, 
    container=list, 
    func=toolbox.individual
  )
  toolbox.register(
    "evaluate", 
    evaluate, 
    curr_temp_list=current_temperature_list, 
    demand_total_time=demand_total_time, 
    supply_granularity=supply_granularity,
    valve_modulation_limit_list=valve_modulation_limit_list, 
    temperature_prediction_coef_list=temperature_prediction_coef_list, 
    num_radiators=num_radiators,
    valve_modulation_energy_consumption_list=valve_modulation_energy_consumption_list, 
    demanded_temperature_list=demanded_temperature_list,
    boiler_inflection_point_list=boiler_inflection_point_list, 
    boiler_cost_list=boiler_cost_list
  )
  
  pop = [creator.Individual(ind) for ind in np.concatenate(pop_list, axis=1)]
  
  # Add OR solution to the population
#   pop[0] = creator.Individual(or_set_modulation)

  hof = tools.HallOfFame(1, similar=np.array_equal)
  
  print(f"All radiators:")
  pop, log = algorithms.eaMuPlusLambda(
    pop, 
    toolbox, 
    mu=population_size, 
    lambda_=offspring_size, 
    cxpb=crossover_proba, 
    mutpb=mutation_proba, 
    ngen=generation_length,
    stats=stats, 
    halloffame=hof, 
    verbose=False
  )
  end_time = time.time()
  
  res_dict["final_log"] = log
  res_dict["runtime"] = end_time - start_time

  recommended_schedule_list = hof[0]
  
  recommended_schedule_evaluation = evaluate(
    recommended_schedule_list, 
    curr_temp_list=current_temperature_list, 
    demand_total_time=demand_total_time,
    supply_granularity=supply_granularity, 
    valve_modulation_limit_list=valve_modulation_limit_list, 
    temperature_prediction_coef_list=temperature_prediction_coef_list, 
    num_radiators=num_radiators,
    valve_modulation_energy_consumption_list=valve_modulation_energy_consumption_list, 
    demanded_temperature_list=demanded_temperature_list,
    boiler_inflection_point_list=boiler_inflection_point_list, 
    boiler_cost_list=boiler_cost_list
  )
  (
    recommended_schedule_temperature_prediction_list, 
    recommended_schedule_energy_consumption_list
  ) = predict_temperature_with_regression(
    recommended_schedule_list,
    current_temperature_list, 
    demand_total_time, 
    supply_granularity, 
    valve_modulation_limit_list, 
    temperature_prediction_coef_list,
    num_radiators, 
    valve_modulation_energy_consumption_list
  )
  recommended_schedule_unmet_demand_list = calculate_unmet_demand(
    recommended_schedule_temperature_prediction_list, 
    demanded_temperature_list
  )
  recommended_schedule_total_energy_consumption_list = recommended_schedule_energy_consumption_list.sum(axis=0)
  recommended_schedule_boiler_energy_consumption_cost_list = np.where(
    recommended_schedule_total_energy_consumption_list <= boiler_inflection_point_list[0],
    boiler_cost_list[0], 
    np.where(
      recommended_schedule_total_energy_consumption_list <= boiler_inflection_point_list[1], 
      boiler_cost_list[1],
      boiler_cost_list[2]
    )
  )
  
  res_dict["recommended_schedule"] = {
    "evaluation_result": recommended_schedule_evaluation[0],
    "negative_unmet_demand_mean": calculate_negative_unmet_demand(recommended_schedule_unmet_demand_list).mean(),
    "energy_consumption_sum": recommended_schedule_boiler_energy_consumption_cost_list.sum(),
    "energy_consumption_mean": recommended_schedule_boiler_energy_consumption_cost_list.mean(),
    "list": recommended_schedule_list,
    "temperature_prediction_list": recommended_schedule_temperature_prediction_list,
    "unmet_demand_list": recommended_schedule_unmet_demand_list,
    "energy_consumption_list": recommended_schedule_boiler_energy_consumption_cost_list
  }
  
  return res_dict, pop

# GA Config

In [3]:
GENERATION_LENGTH = 20
POPULATION_SIZE = 500
OFFSPRING_SIZE = 250


CROSSOVER_PROBA = 0.5
CX_APARTMENT_PROBA = 1
CX_TIME_POINT_PROBA = 0.5


MUTATION_PROBA = 0.5
MUTATION_APARTMENT_PROBA = 1
MUTATION_TIME_POINT_PROBA = 0.1


DATA_UNIT_TIME = 300
DEMAND_GRANULARITY = 12
SUPPLY_GRANULARITY = 1


VALVE_MIN_MODULATION = 0
VALVE_MAX_MODULATION = 3
VALVE_MODULATION_LIMITS = np.array([0.1, 0.5, 1.5])
VALVE_MODULATION_ENERGY_CONSUMPTIONS = np.array([0.0166, 0.0630, 0.0824, 0.1046])
BOILER_COSTS = np.array([5.0, 15.0, 40.0])

# Data Generation

In [4]:
num_radiators = 3
application_period_in_hours = 3


boiler_inflection_point_list = get_boiler_inflection_points(num_radiators, VALVE_MODULATION_ENERGY_CONSUMPTIONS[-1])
temperature_prediction_coef_list = get_temperature_prediction_coefficients(num_radiators)


current_temperature_list = np.array(
  [18.78488148, 18.73217506, 19.64874133]
)


demanded_temperature_list = np.array(
  [
    [19., 20., 21.],
    [19., 18.5, 21],
    [19., 21., 18.]
  ]
)

# Batch Data Generation

In [5]:
INITIAL_TEMP_MIN = 18.0
INITIAL_TEMP_MAX = 20.0
DEMANDED_TEMP_STEP = 0.5


def generate_initial_temperature_data(num_radiators):
  return np.random.uniform(
    INITIAL_TEMP_MIN, 
    INITIAL_TEMP_MAX, 
    num_radiators
  )


# def generate_demanded_temperature_data(initial_temperature_data, application_period_in_hours):
#   demanded_temperature_data = []
#   for initial_radiator_temperature in initial_temperature_data:
#     radiator_demanded_temperature_data = [round(initial_radiator_temperature * 2) / 2]
#     for _ in range(application_period_in_hours):
#       curr_demand = radiator_demanded_temperature_data[-1]
#       next_demand = np.random.choice(
#         np.arange(curr_demand - 0.5, curr_demand + 1, DEMANDED_TEMP_STEP)
#       )
#       next_demand = np.clip(next_demand, DEMANDED_TEMP_MIN, DEMANDED_TEMP_MAX)
#       radiator_demanded_temperature_data.append(next_demand)
#     demanded_temperature_data.append(radiator_demanded_temperature_data[1:])
#   return np.array(demanded_temperature_data)


def generate_demanded_temperature_data(num_radiators, application_period_in_hours):
  demanded_temperatures_1 = np.random.choice(
    np.arange(
      18.5, 
      19.5 + 1e-10, 
      DEMANDED_TEMP_STEP
    ), 
    size=(num_radiators, application_period_in_hours // 3)
  )
  
  demanded_temperatures_2 = np.random.choice(
    np.arange(
      19.5, 
      23 + 1e-10, 
      DEMANDED_TEMP_STEP
    ), 
    size=(num_radiators, application_period_in_hours // 3)
  )
  
  demanded_temperatures_3 = np.random.choice(
    np.arange(
      19, 
      21 + 1e-10, 
      DEMANDED_TEMP_STEP
    ), 
    size=(num_radiators, application_period_in_hours // 3)
  )
  
  return np.concatenate(
    (
      demanded_temperatures_1, 
      demanded_temperatures_2,
      demanded_temperatures_3
    ), 
    axis=1
  )

In [ ]:
num_radiators_space = [3, 5, 10]
application_period_in_hours_space = [6, 12]


synth_data = {}
for num_radiators in num_radiators_space:
  for application_period_in_hours in application_period_in_hours_space:
    boiler_inflection_point_list = get_boiler_inflection_points(
      num_radiators, 
      VALVE_MODULATION_ENERGY_CONSUMPTIONS[-1]
    )
    temperature_prediction_coef_list = get_temperature_prediction_coefficients(
      num_radiators
    )
    
    print(f'({num_radiators}, {application_period_in_hours})')
    
    trial_counter = 0
    mean_unmet_demand = -1
    while mean_unmet_demand < 0:
      trial_counter += 1
      print('\t' + f'Trial count: {trial_counter}')
      
      init_temp_data = generate_initial_temperature_data(num_radiators)
      demanded_temp_data = generate_demanded_temperature_data(num_radiators, application_period_in_hours)
      
      mean_unmet_demand = genetic_solver_run(
        num_radiators, 
        POPULATION_SIZE, 
        OFFSPRING_SIZE, 
        CROSSOVER_PROB, 
        MUTATION_PROB, 
        IND_MUTATION_PROB, 
        DEMAND_GRANULARITY, 
        SUPPLY_GRANULARITY, 
        application_period_in_hours, 
        DATA_UNIT_TIME, 
        GENERATION_LENGTH, 
        VALVE_MIN_MODULATION,
        VALVE_MAX_MODULATION,  
        init_temp_data, 
        demanded_temp_data, 
        VALVE_MODULATION_LIMITS, 
        VALVE_MODULATION_ENERGY_CONSUMPTIONS, 
        boiler_inflection_point_list, 
        BOILER_COSTS,
        temperature_prediction_coef_list
      )

    synth_data[(num_radiators, application_period_in_hours)] = {
      'init_temp': init_temp_data,
      'demanded_temp': demanded_temp_data
    }
    
    print('Initial temperatures:')
    print(init_temp_data)
    print('Demanded temperatures')
    print(demanded_temp_data)
    print()
    print()

joblib.dump(synth_data, 'paper_synth_data.pk')
synth_data

# Batch Run

In [4]:
synth_batch_data = joblib.load('paper_synth_data.pk')
total_num_run = 1
res_dict_filename = "res_dict_5000_gen.pk"

In [5]:
res_dict = dict()
for k, v in synth_batch_data.items():
  num_radiators, application_period_in_hours = k
  current_temperature_list = v['init_temp']
  demanded_temperature_list = v['demanded_temp']
  
  boiler_inflection_point_list = get_boiler_inflection_points(
    num_radiators, 
    VALVE_MODULATION_ENERGY_CONSUMPTIONS[-1]
  )
  temperature_prediction_coef_list = get_temperature_prediction_coefficients(
    num_radiators
  )
  
  res_dict[(num_radiators, application_period_in_hours)] = {
    "initial_temperatures": current_temperature_list,
    "demanded_temperatures": demanded_temperature_list
  }
  
  print()
  print('#' * 100)
  print()
  print(f'Number of radiators: {num_radiators}')
  print(f'Application period in hours: {application_period_in_hours}')
  
  for num_run in range(1, total_num_run + 1):
    print()
    print(f'Run number: {num_run}')

    run_res_dict, _ = genetic_solver_run(
      num_radiators, 
      POPULATION_SIZE, 
      OFFSPRING_SIZE, 
      CROSSOVER_PROBA,
      CX_APARTMENT_PROBA,
      CX_TIME_POINT_PROBA,
      MUTATION_PROBA,
      MUTATION_APARTMENT_PROBA,
      MUTATION_TIME_POINT_PROBA,
      DEMAND_GRANULARITY, 
      SUPPLY_GRANULARITY, 
      application_period_in_hours, 
      DATA_UNIT_TIME, 
      GENERATION_LENGTH, 
      VALVE_MIN_MODULATION,
      VALVE_MAX_MODULATION, 
      current_temperature_list, 
      demanded_temperature_list, 
      VALVE_MODULATION_LIMITS, 
      VALVE_MODULATION_ENERGY_CONSUMPTIONS, 
      boiler_inflection_point_list, 
      BOILER_COSTS,
      temperature_prediction_coef_list
    )
    
    res_dict[(num_radiators, application_period_in_hours)][num_run] = run_res_dict
    joblib.dump(res_dict, res_dict_filename)
    
    print(
      "\tCurrent practice energy consumption:", 
      res_dict[k][num_run]["current_practice"]["energy_consumption_sum"]
    )
    print(
      "\tRecommended schedule energy consumption:", 
      res_dict[k][num_run]["recommended_schedule"]["energy_consumption_sum"]
    )
    print(
      "\tGA runtime:", 
      res_dict[k][num_run]["runtime"]
    )


####################################################################################################

Number of radiators: 3
Application period in hours: 6

Run number: 1
Radiators 1, 2, 3


100%|███████████████████████████████████████████████████████████████| 20/20 [00:05<00:00,  3.71it/s]


All radiators:


100%|███████████████████████████████████████████████████████████████| 20/20 [00:05<00:00,  3.81it/s]


	Current practice energy consumption: 715.0
	Recommended schedule energy consumption: 1025.0
	GA runtime: 11.640002250671387

####################################################################################################

Number of radiators: 3
Application period in hours: 12

Run number: 1
Radiators 1, 2, 3


...

All radiators:


100%|███████████████████████████████████████████████████████████████| 20/20 [00:11<00:00,  1.79it/s]


	Current practice energy consumption: 915.0
	Recommended schedule energy consumption: 2125.0
	GA runtime: 23.30772066116333

####################################################################################################

Number of radiators: 5
Application period in hours: 6

Run number: 1
Radiators 1, 2, 3


  5%|███▏                                                            | 1/20 [00:00<00:09,  1.92it/s]


KeyboardInterrupt: 

In [31]:
experiment = (3, 6)
or_objective = 510


if "res_dict" not in locals().keys():
  res_dict = joblib.load(res_dict_filename)


print(
  "Experiment:", experiment, "\n"
)
for num_run in range(1, total_num_run + 1):
  print(
    "Run number:", num_run
  )
  print(
    "\tCurrent practice energy consumption:", 
    res_dict[experiment][num_run]["current_practice"]["energy_consumption_sum"]
  )
  print(
    "\tRecommended schedule energy consumption:", 
    res_dict[experiment][num_run]["recommended_schedule"]["energy_consumption_sum"]
  )
  print(
    "\tGA runtime:", 
    res_dict[experiment][num_run]["runtime"]
  )
  
clipboard.copy(
  "\t".join(
    [str(res_dict[experiment][1]["current_practice"]["energy_consumption_sum"])] + \
    [
      str(res_dict[experiment][num_run]["recommended_schedule"]["energy_consumption_sum"]) + \
      "\t" + \
      str(res_dict[experiment][num_run]["runtime"]) + 
      "\t" + \
      str(
        abs(
          or_objective - \
          res_dict[experiment][num_run]["recommended_schedule"]["energy_consumption_sum"]
        ) / \
        or_objective * 100
      )
      for num_run
      in range(1, total_num_run + 1)
    ]
  )
)

Experiment: (3, 6) 

Run number: 1
	Current practice energy consumption: 715.0
	Recommended schedule energy consumption: 525.0
	GA runtime: 3307.9134788513184
Run number: 2
	Current practice energy consumption: 715.0
	Recommended schedule energy consumption: 510.0
	GA runtime: 3582.4292109012604
Run number: 3
	Current practice energy consumption: 715.0
	Recommended schedule energy consumption: 510.0
	GA runtime: 3559.873739004135
Run number: 4
	Current practice energy consumption: 715.0
	Recommended schedule energy consumption: 510.0
	GA runtime: 3621.4103157520294
Run number: 5
	Current practice energy consumption: 715.0
	Recommended schedule energy consumption: 525.0
	GA runtime: 3623.44708108902


# Single Run

In [5]:
num_radiators = 3
application_period_in_hours = 6
print(f'Number of radiators: {num_radiators}')
print(f'Application period in hours: {application_period_in_hours}')

Number of radiators: 3
Application period in hours: 6


In [6]:
synth_batch_data = joblib.load('paper_synth_data.pk')
v = synth_batch_data[(num_radiators, application_period_in_hours)]

In [7]:
boiler_inflection_point_list = get_boiler_inflection_points(
  num_radiators, 
  VALVE_MODULATION_ENERGY_CONSUMPTIONS[-1]
)
temperature_prediction_coef_list = get_temperature_prediction_coefficients(
  num_radiators
)

current_temperature_list = v['init_temp']
demanded_temperature_list = v['demanded_temp']
print('Initial temperatures:')
print(current_temperature_list)
print('Demanded temperatures:')
print(demanded_temperature_list)
print('#' * 100)
print()

Initial temperatures:
[18.78488148 18.73217506 19.64874133]
Demanded temperatures:
[[19.  19.  20.  20.5 19.  19. ]
 [19.  18.5 22.5 20.5 19.  19.5]
 [19.  18.5 23.  23.  19.5 19. ]]
####################################################################################################



In [5]:
# or_set_temp = np.array(joblib.load(f'OR_set_temp/s_{num_radiators}{application_period_in_hours}.pkl'))
# or_set_temp = or_set_temp[:, :-1]
# print(or_set_temp.shape)
# or_set_temp

(3, 72)


array([[19.28488148, 19.22396173, 19.2199738 , 19.22598587, 19.21199794,
        19.56107819, 19.20402207, 19.20003414, 19.19604621, 19.19205828,
        19.18807034, 19.18408241, 19.18009448, 19.17610655, 19.17211862,
        19.16813069, 19.86027912, 19.16015482, 19.15616689, 19.15217896,
        19.14819103, 19.14420309, 19.14021516, 19.38555814, 20.17144384,
        20.1       , 20.1       , 20.1       , 20.1       , 20.1       ,
        20.1       , 20.1       , 20.1       , 20.1       , 20.1       ,
        20.1       , 20.46665684, 20.6       , 20.6       , 20.6       ,
        20.6       , 20.6       , 20.6       , 20.6       , 20.6       ,
        20.6       , 20.6       , 20.6       , 20.6       , 20.59601207,
        20.59202414, 20.5880362 , 20.58404827, 20.58006034, 20.57607241,
        20.57208448, 20.56809655, 20.56410861, 20.56012068, 20.55613275,
        20.55214482, 20.54815689, 20.54416895, 20.54018102, 20.53619309,
        20.53220516, 20.52821723, 20.5242293 , 20.5

In [7]:
ga_set_temp, pop = genetic_solver_run(
  num_radiators, 
  POPULATION_SIZE, 
  OFFSPRING_SIZE, 
  CROSSOVER_PROBA,
  CX_APARTMENT_PROBA,
  CX_TIME_POINT_PROBA,
  MUTATION_PROBA,
  MUTATION_APARTMENT_PROBA,
  MUTATION_TIME_POINT_PROBA,
  DEMAND_GRANULARITY, 
  SUPPLY_GRANULARITY, 
  application_period_in_hours, 
  DATA_UNIT_TIME, 
  GENERATION_LENGTH, 
  VALVE_MIN_MODULATION,
  VALVE_MAX_MODULATION, 
  current_temperature_list, 
  demanded_temperature_list, 
  VALVE_MODULATION_LIMITS, 
  VALVE_MODULATION_ENERGY_CONSUMPTIONS, 
  boiler_inflection_point_list, 
  BOILER_COSTS,
  temperature_prediction_coef_list
)

Current practice evaluation result: 744257.4551638521
Current practice negative unmet demand mean: -0.14669575620229688
Current practice energy consumption sum: 715.0
Current practice energy consumption mean: 9.930555555555555
Demanded temperature list:
[[19.  19.  19.  19.  19.  19.  19.  19.  19.  19.  19.  19.  19.  19.
  19.  19.  19.  19.  19.  19.  19.  19.  19.  19.  20.  20.  20.  20.
  20.  20.  20.  20.  20.  20.  20.  20.  20.5 20.5 20.5 20.5 20.5 20.5
  20.5 20.5 20.5 20.5 20.5 20.5 19.  19.  19.  19.  19.  19.  19.  19.
  19.  19.  19.  19.  19.  19.  19.  19.  19.  19.  19.  19.  19.  19.
  19.  19. ]
 [19.  19.  19.  19.  19.  19.  19.  19.  19.  19.  19.  19.  18.5 18.5
  18.5 18.5 18.5 18.5 18.5 18.5 18.5 18.5 18.5 18.5 22.5 22.5 22.5 22.5
  22.5 22.5 22.5 22.5 22.5 22.5 22.5 22.5 20.5 20.5 20.5 20.5 20.5 20.5
  20.5 20.5 20.5 20.5 20.5 20.5 19.  19.  19.  19.  19.  19.  19.  19.
  19.  19.  19.  19.  19.5 19.5 19.5 19.5 19.5 19.5 19.5 19.5 19.5 19.5
  19.5 19.5]
 [19.

gen	nevals	avg    	std    	min 	max   
0  	1000  	4373.22	6359.74	1675	111898
1  	600   	2807.36	2206.06	1675	46782.9
####################################################################################################
#####                                                                                          #####
#####                              Combining Subproblem Solutions                              #####
#####                                                                                          #####
####################################################################################################
gen	nevals	avg    	std    	min 	max    
0  	999   	2807.36	2206.06	1675	46782.9
1  	600   	2303.91	1795.46	1675	46782.9
Run-time: 5.072342872619629
Recommended schedule evaluation result: 1675.0
Recommended schedule negative unmet demand mean: 0.0
Recommended schedule energy consumption sum: 1675.0
Recommended schedule energy consumption mean: 23.26388888888889
Demanded temp

In [39]:
evaluate(
  ga_set_temp, 
  curr_temp_list=current_temperature_list, 
  demand_total_time=int(application_period_in_hours * DEMAND_GRANULARITY),
  supply_granularity=SUPPLY_GRANULARITY, 
  valve_modulation_limit_list=VALVE_MODULATION_LIMITS, 
  temperature_prediction_coef_list=temperature_prediction_coef_list, 
  num_radiators=num_radiators,
  valve_modulation_energy_consumption_list=VALVE_MODULATION_ENERGY_CONSUMPTIONS, 
  demanded_temperature_list=np.repeat(
    demanded_temperature_list, 
    DEMAND_GRANULARITY
  ).reshape(num_radiators, -1),
  boiler_inflection_point_list=boiler_inflection_point_list, 
  boiler_cost_list=BOILER_COSTS
)

(850.0,)

In [45]:
(
  ga_set_temp_temperature_prediction_list, 
  ga_set_temp_energy_consumption_list
) = predict_temperature_with_regression(
  ga_set_temp,
  current_temperature_list, 
  int(application_period_in_hours * DEMAND_GRANULARITY), 
  SUPPLY_GRANULARITY, 
  VALVE_MODULATION_LIMITS, 
  temperature_prediction_coef_list,
  num_radiators, 
  VALVE_MODULATION_ENERGY_CONSUMPTIONS
)
ga_set_temp_unmet_demand_list = calculate_unmet_demand(
  ga_set_temp_temperature_prediction_list, 
  np.repeat(
    demanded_temperature_list, 
    DEMAND_GRANULARITY
  ).reshape(num_radiators, -1)
)
ga_set_temp_total_energy_consumption_list = ga_set_temp_energy_consumption_list.sum(axis=0)
ga_set_temp_boiler_energy_consumption_cost_list = np.where(
  ga_set_temp_total_energy_consumption_list <= boiler_inflection_point_list[0],
  BOILER_COSTS[0], 
  np.where(
    ga_set_temp_total_energy_consumption_list <= boiler_inflection_point_list[1], 
    BOILER_COSTS[1],
    BOILER_COSTS[2]
  )
)
print(f"GA set temp negative unmet demand mean: {calculate_negative_unmet_demand(ga_set_temp_unmet_demand_list).mean()}")
print(f"GA set temp energy consumption sum: {ga_set_temp_boiler_energy_consumption_cost_list.sum()}")
print(f"GA set temp energy consumption mean: {ga_set_temp_boiler_energy_consumption_cost_list.mean()}")

GA set temp negative unmet demand mean: 0.0
GA set temp energy consumption sum: 720.0
GA set temp energy consumption mean: 10.0
